In [ ]:
import torch
from dataset_utils import edge_stratified_split
from classification_dataset import ClassificationDataset
from  config.parser_config import config_parser
from torch.utils.data import DataLoader

In [ ]:
parser = config_parser()
argv = '--config input_config'.split() 
args  = parser.parse_args(argv)
full_dataset = ClassificationDataset(
        one_hot = False,
        augmentation= 'no augment',
        npz_path= args.npz_path,
        image_path= args.image_path,
        label_path= args.label_path,
        size = args.size,
        normalize= True)

edge_classes = ["Gryllteiste","Schnatterente","Buchfink","unbestimmte Larusmöwe",
                    "Schmarotzer/Spatel/Falkenraubmöwe","Brandgans","Wasserlinie mit Großalgen",
                    "Feldlerche","Schmarotzerraubmöwe","Grosser Brachvogel","unbestimmte Raubmöwe",
                    "Turmfalke","Trauerseeschwalbe","unbestimmter Schwan",
                    "Sperber","Kiebitzregenpfeifer",
                    "Skua","Graugans","unbestimmte Krähe"]

edge_labels = [full_dataset._get_label_from_cat(cat) for cat in edge_classes]

edge_train_data, _ = edge_stratified_split(full_dataset, full_labels = full_dataset._labels, edge_labels = edge_labels,  fraction = 0.8, random_state = 0)                     
loader = DataLoader(edge_train_data,
                                batch_size=8, 
                                shuffle=True,
                                num_workers=args.num_workers)

In [1]:
edge_classes = ["Gryllteiste","Schnatterente","Buchfink","unbestimmte Larusmöwe",
                        "Schmarotzer/Spatel/Falkenraubmöwe","Brandgans","Wasserlinie mit Großalgen",
                        "Feldlerche","Schmarotzerraubmöwe","Grosser Brachvogel","unbestimmte Raubmöwe",
                        "Turmfalke","Trauerseeschwalbe","unbestimmter Schwan",
                        "Sperber","Kiebitzregenpfeifer",
                        "Skua","Graugans","unbestimmte Krähe"]


In [2]:
import torch

/home/student/anaconda3/envs/efficientnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
